In [1]:

import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

120
500
1000


# GCN using only domain Feature

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [20]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
pred=torch.tensor([1, 0, 1,0])
label=torch.tensor([1, 1, 1,0])
correct = pred.eq(label).sum().item()
total=len(label)
print(correct)
print(total)
print(correct/total)
print(ACC(pred,label))

In [6]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.0083, Train: 83.33%, Valid: 51.80% Test: 50.80%
Run: 01, Epoch: 02, Loss: 1.0460, Train: 91.67%, Valid: 59.40% Test: 58.50%
Run: 01, Epoch: 03, Loss: 0.6907, Train: 95.00%, Valid: 62.40% Test: 60.30%
Run: 01, Epoch: 04, Loss: 0.5743, Train: 97.50%, Valid: 64.40% Test: 60.70%
Run: 01, Epoch: 05, Loss: 0.4541, Train: 97.50%, Valid: 63.00% Test: 60.20%
Run: 01, Epoch: 06, Loss: 0.3901, Train: 97.50%, Valid: 61.80% Test: 58.80%
Run: 01, Epoch: 07, Loss: 0.3025, Train: 97.50%, Valid: 60.20% Test: 58.20%
Run: 01, Epoch: 08, Loss: 0.3009, Train: 99.17%, Valid: 58.00% Test: 57.20%
Run: 01, Epoch: 09, Loss: 0.2813, Train: 99.17%, Valid: 57.00% Test: 55.70%
Run: 01, Epoch: 10, Loss: 0.2018, Train: 99.17%, Valid: 55.60% Test: 54.30%
Run: 01, Epoch: 11, Loss: 0.1896, Train: 99.17%, Valid: 54.60% Test: 53.90%
Run: 01, Epoch: 12, Loss: 0.1690, Train: 100.00%, Valid: 53.40% Test: 53.50%
Run: 01, Epoch: 13, Loss: 0.1939, Train: 100.00%, Valid: 53.00% Test: 52.50%
Run: 01, E

Run: 01, Epoch: 117, Loss: 0.0193, Train: 100.00%, Valid: 44.20% Test: 45.20%
Run: 01, Epoch: 118, Loss: 0.0402, Train: 100.00%, Valid: 44.20% Test: 45.40%
Run: 01, Epoch: 119, Loss: 0.0172, Train: 100.00%, Valid: 44.40% Test: 45.50%
Run: 01, Epoch: 120, Loss: 0.0290, Train: 100.00%, Valid: 44.40% Test: 45.40%
Run: 01, Epoch: 121, Loss: 0.0158, Train: 100.00%, Valid: 44.40% Test: 45.40%
Run: 01, Epoch: 122, Loss: 0.0466, Train: 100.00%, Valid: 44.40% Test: 45.60%
Run: 01, Epoch: 123, Loss: 0.0123, Train: 100.00%, Valid: 44.60% Test: 45.40%
Run: 01, Epoch: 124, Loss: 0.0246, Train: 100.00%, Valid: 44.20% Test: 45.50%
Run: 01, Epoch: 125, Loss: 0.0123, Train: 100.00%, Valid: 44.00% Test: 45.40%
Run: 01, Epoch: 126, Loss: 0.0088, Train: 100.00%, Valid: 43.40% Test: 45.80%
Run: 01, Epoch: 127, Loss: 0.0125, Train: 100.00%, Valid: 43.40% Test: 45.70%
Run: 01, Epoch: 128, Loss: 0.0042, Train: 100.00%, Valid: 43.20% Test: 45.70%
Run: 01, Epoch: 129, Loss: 0.0152, Train: 100.00%, Valid: 43.20%

Run: 02, Epoch: 32, Loss: 0.0867, Train: 100.00%, Valid: 50.80% Test: 48.90%
Run: 02, Epoch: 33, Loss: 0.0588, Train: 100.00%, Valid: 50.80% Test: 48.40%
Run: 02, Epoch: 34, Loss: 0.0499, Train: 100.00%, Valid: 50.80% Test: 48.30%
Run: 02, Epoch: 35, Loss: 0.0589, Train: 100.00%, Valid: 51.00% Test: 48.50%
Run: 02, Epoch: 36, Loss: 0.0462, Train: 100.00%, Valid: 51.00% Test: 48.40%
Run: 02, Epoch: 37, Loss: 0.0734, Train: 100.00%, Valid: 51.00% Test: 48.50%
Run: 02, Epoch: 38, Loss: 0.0370, Train: 100.00%, Valid: 50.80% Test: 48.20%
Run: 02, Epoch: 39, Loss: 0.0576, Train: 100.00%, Valid: 51.00% Test: 48.10%
Run: 02, Epoch: 40, Loss: 0.0328, Train: 100.00%, Valid: 50.80% Test: 48.40%
Run: 02, Epoch: 41, Loss: 0.0494, Train: 100.00%, Valid: 50.80% Test: 48.40%
Run: 02, Epoch: 42, Loss: 0.0368, Train: 100.00%, Valid: 50.80% Test: 48.10%
Run: 02, Epoch: 43, Loss: 0.0410, Train: 100.00%, Valid: 50.60% Test: 48.10%
Run: 02, Epoch: 44, Loss: 0.0476, Train: 100.00%, Valid: 50.40% Test: 48.10%

Run: 02, Epoch: 144, Loss: 0.0154, Train: 100.00%, Valid: 46.60% Test: 45.40%
Run: 02, Epoch: 145, Loss: 0.0053, Train: 100.00%, Valid: 46.60% Test: 45.40%
Run: 02, Epoch: 146, Loss: 0.0110, Train: 100.00%, Valid: 46.60% Test: 45.30%
Run: 02, Epoch: 147, Loss: 0.0165, Train: 100.00%, Valid: 46.40% Test: 45.20%
Run: 02, Epoch: 148, Loss: 0.0062, Train: 100.00%, Valid: 46.40% Test: 45.10%
Run: 02, Epoch: 149, Loss: 0.0167, Train: 100.00%, Valid: 46.40% Test: 45.00%
Run: 02, Epoch: 150, Loss: 0.0148, Train: 100.00%, Valid: 46.20% Test: 44.80%
Run: 02, Epoch: 151, Loss: 0.0272, Train: 100.00%, Valid: 45.80% Test: 45.00%
Run: 02, Epoch: 152, Loss: 0.0150, Train: 100.00%, Valid: 45.40% Test: 45.10%
Run: 02, Epoch: 153, Loss: 0.0073, Train: 100.00%, Valid: 45.40% Test: 45.00%
Run: 02, Epoch: 154, Loss: 0.0119, Train: 100.00%, Valid: 45.40% Test: 45.10%
Run: 02, Epoch: 155, Loss: 0.0276, Train: 100.00%, Valid: 45.20% Test: 45.10%
Run: 02, Epoch: 156, Loss: 0.0188, Train: 100.00%, Valid: 45.20%

Run: 03, Epoch: 62, Loss: 0.0350, Train: 100.00%, Valid: 46.80% Test: 47.30%
Run: 03, Epoch: 63, Loss: 0.0398, Train: 100.00%, Valid: 46.80% Test: 47.40%
Run: 03, Epoch: 64, Loss: 0.0583, Train: 100.00%, Valid: 46.60% Test: 47.50%
Run: 03, Epoch: 65, Loss: 0.0542, Train: 100.00%, Valid: 46.60% Test: 47.50%
Run: 03, Epoch: 66, Loss: 0.0463, Train: 100.00%, Valid: 46.80% Test: 47.40%
Run: 03, Epoch: 67, Loss: 0.0556, Train: 100.00%, Valid: 47.00% Test: 47.40%
Run: 03, Epoch: 68, Loss: 0.0399, Train: 100.00%, Valid: 47.20% Test: 47.50%
Run: 03, Epoch: 69, Loss: 0.0340, Train: 100.00%, Valid: 47.20% Test: 47.40%
Run: 03, Epoch: 70, Loss: 0.0203, Train: 100.00%, Valid: 47.20% Test: 47.30%
Run: 03, Epoch: 71, Loss: 0.0360, Train: 100.00%, Valid: 47.20% Test: 47.40%
Run: 03, Epoch: 72, Loss: 0.0243, Train: 100.00%, Valid: 46.80% Test: 47.60%
Run: 03, Epoch: 73, Loss: 0.0575, Train: 100.00%, Valid: 46.60% Test: 47.50%
Run: 03, Epoch: 74, Loss: 0.0654, Train: 100.00%, Valid: 46.00% Test: 47.20%

...

Run: 08, Epoch: 61, Loss: 0.0326, Train: 100.00%, Valid: 49.00% Test: 49.10%
Run: 08, Epoch: 62, Loss: 0.0462, Train: 100.00%, Valid: 49.40% Test: 49.50%
Run: 08, Epoch: 63, Loss: 0.0364, Train: 100.00%, Valid: 49.40% Test: 49.30%
Run: 08, Epoch: 64, Loss: 0.0610, Train: 100.00%, Valid: 49.40% Test: 49.20%
Run: 08, Epoch: 65, Loss: 0.0398, Train: 100.00%, Valid: 49.40% Test: 48.80%
Run: 08, Epoch: 66, Loss: 0.0278, Train: 100.00%, Valid: 49.40% Test: 48.90%
Run: 08, Epoch: 67, Loss: 0.0301, Train: 100.00%, Valid: 49.40% Test: 48.80%
Run: 08, Epoch: 68, Loss: 0.0134, Train: 100.00%, Valid: 49.80% Test: 48.80%
Run: 08, Epoch: 69, Loss: 0.0129, Train: 100.00%, Valid: 50.00% Test: 48.90%
Run: 08, Epoch: 70, Loss: 0.0148, Train: 100.00%, Valid: 49.80% Test: 49.10%
Run: 08, Epoch: 71, Loss: 0.0210, Train: 100.00%, Valid: 50.00% Test: 49.10%
Run: 08, Epoch: 72, Loss: 0.0264, Train: 100.00%, Valid: 49.80% Test: 49.00%
Run: 08, Epoch: 73, Loss: 0.0130, Train: 100.00%, Valid: 49.80% Test: 48.90%

Run: 08, Epoch: 170, Loss: 0.0170, Train: 100.00%, Valid: 47.60% Test: 46.80%
Run: 08, Epoch: 171, Loss: 0.0456, Train: 100.00%, Valid: 47.60% Test: 46.70%
Run: 08, Epoch: 172, Loss: 0.0216, Train: 100.00%, Valid: 47.80% Test: 46.60%
Run: 08, Epoch: 173, Loss: 0.0134, Train: 100.00%, Valid: 48.00% Test: 46.50%
Run: 08, Epoch: 174, Loss: 0.0108, Train: 100.00%, Valid: 48.40% Test: 46.60%
Run: 08, Epoch: 175, Loss: 0.0215, Train: 100.00%, Valid: 48.40% Test: 46.40%
Run: 08, Epoch: 176, Loss: 0.0310, Train: 100.00%, Valid: 47.80% Test: 46.40%
Run: 08, Epoch: 177, Loss: 0.0045, Train: 100.00%, Valid: 48.00% Test: 46.50%
Run: 08, Epoch: 178, Loss: 0.0263, Train: 100.00%, Valid: 48.00% Test: 46.50%
Run: 08, Epoch: 179, Loss: 0.0103, Train: 100.00%, Valid: 48.00% Test: 46.30%
Run: 08, Epoch: 180, Loss: 0.0153, Train: 100.00%, Valid: 48.20% Test: 46.10%
Run: 08, Epoch: 181, Loss: 0.0043, Train: 100.00%, Valid: 48.20% Test: 46.20%
Run: 08, Epoch: 182, Loss: 0.0127, Train: 100.00%, Valid: 48.40%

Run: 09, Epoch: 85, Loss: 0.0179, Train: 100.00%, Valid: 51.20% Test: 50.10%
Run: 09, Epoch: 86, Loss: 0.0418, Train: 100.00%, Valid: 51.00% Test: 50.30%
Run: 09, Epoch: 87, Loss: 0.0233, Train: 100.00%, Valid: 51.00% Test: 50.10%
Run: 09, Epoch: 88, Loss: 0.0203, Train: 100.00%, Valid: 51.00% Test: 50.30%
Run: 09, Epoch: 89, Loss: 0.0158, Train: 100.00%, Valid: 51.00% Test: 50.40%
Run: 09, Epoch: 90, Loss: 0.0262, Train: 100.00%, Valid: 51.00% Test: 50.40%
Run: 09, Epoch: 91, Loss: 0.0467, Train: 100.00%, Valid: 51.00% Test: 50.40%
Run: 09, Epoch: 92, Loss: 0.0296, Train: 100.00%, Valid: 51.40% Test: 50.30%
Run: 09, Epoch: 93, Loss: 0.0365, Train: 100.00%, Valid: 51.00% Test: 50.20%
Run: 09, Epoch: 94, Loss: 0.0249, Train: 100.00%, Valid: 51.00% Test: 50.30%
Run: 09, Epoch: 95, Loss: 0.0369, Train: 100.00%, Valid: 51.20% Test: 50.40%
Run: 09, Epoch: 96, Loss: 0.0080, Train: 100.00%, Valid: 51.20% Test: 50.50%
Run: 09, Epoch: 97, Loss: 0.0309, Train: 100.00%, Valid: 51.20% Test: 50.40%

Run: 09, Epoch: 195, Loss: 0.0302, Train: 100.00%, Valid: 49.00% Test: 49.60%
Run: 09, Epoch: 196, Loss: 0.0274, Train: 100.00%, Valid: 49.00% Test: 49.60%
Run: 09, Epoch: 197, Loss: 0.0179, Train: 100.00%, Valid: 49.00% Test: 49.70%
Run: 09, Epoch: 198, Loss: 0.0198, Train: 100.00%, Valid: 48.60% Test: 49.90%
Run: 09, Epoch: 199, Loss: 0.0083, Train: 100.00%, Valid: 48.80% Test: 50.00%
Run: 09, Epoch: 200, Loss: 0.0119, Train: 100.00%, Valid: 48.60% Test: 50.30%
Run 09:
Highest Train: 100.00
Highest Valid: 60.00
  Final Train: 99.17
   Final Test: 59.50
Run: 10, Epoch: 01, Loss: 2.0349, Train: 78.33%, Valid: 33.00% Test: 34.10%
Run: 10, Epoch: 02, Loss: 0.9606, Train: 89.17%, Valid: 43.00% Test: 44.00%
Run: 10, Epoch: 03, Loss: 0.6916, Train: 91.67%, Valid: 49.20% Test: 49.50%
Run: 10, Epoch: 04, Loss: 0.4644, Train: 94.17%, Valid: 52.80% Test: 52.70%
Run: 10, Epoch: 05, Loss: 0.4123, Train: 95.83%, Valid: 54.60% Test: 53.10%
Run: 10, Epoch: 06, Loss: 0.3941, Train: 95.83%, Valid: 55.

Run: 10, Epoch: 104, Loss: 0.0413, Train: 100.00%, Valid: 50.80% Test: 52.80%
Run: 10, Epoch: 105, Loss: 0.0374, Train: 100.00%, Valid: 50.40% Test: 53.00%
Run: 10, Epoch: 106, Loss: 0.0168, Train: 100.00%, Valid: 50.40% Test: 52.80%
Run: 10, Epoch: 107, Loss: 0.0203, Train: 100.00%, Valid: 50.60% Test: 52.60%
Run: 10, Epoch: 108, Loss: 0.0159, Train: 100.00%, Valid: 50.80% Test: 52.60%
Run: 10, Epoch: 109, Loss: 0.0255, Train: 100.00%, Valid: 50.80% Test: 52.70%
Run: 10, Epoch: 110, Loss: 0.0174, Train: 100.00%, Valid: 51.00% Test: 52.50%
Run: 10, Epoch: 111, Loss: 0.0234, Train: 100.00%, Valid: 50.60% Test: 52.60%
Run: 10, Epoch: 112, Loss: 0.0361, Train: 100.00%, Valid: 50.60% Test: 52.40%
Run: 10, Epoch: 113, Loss: 0.0344, Train: 100.00%, Valid: 50.40% Test: 52.50%
Run: 10, Epoch: 114, Loss: 0.0395, Train: 100.00%, Valid: 50.40% Test: 52.70%
Run: 10, Epoch: 115, Loss: 0.0296, Train: 100.00%, Valid: 50.60% Test: 52.90%
Run: 10, Epoch: 116, Loss: 0.0108, Train: 100.00%, Valid: 50.40%

# WISE EMBEDDDING

In [11]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [12]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,3694,3695,3696,3697,3698,3699,3700,3701,3702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [13]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [15]:
def Similarity(array1, array2):
    intersection = np.sum(np.logical_and(array1, array2))
    return intersection
Fec=[]
for i in range(Number_nodes):
#for i in range(2):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    vec.append(Similarity(f,basis[0]))
    vec.append(Similarity(f,basis[1]))
    vec.append(Similarity(f,basis[2]))
    vec.append(Similarity(f,basis[3]))
    vec.append(Similarity(f,basis[4]))
    vec.append(Similarity(f,basis[5]))
    #print(f)
    f.clear()
    Fec.append(vec)
SFec=[]
for i in range(Number_nodes):
#for i in range(2):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    Svec.append(Similarity(f,sel_basis[0]))
    Svec.append(Similarity(f,sel_basis[1]))
    Svec.append(Similarity(f,sel_basis[2]))
    Svec.append(Similarity(f,sel_basis[3]))
    Svec.append(Similarity(f,sel_basis[4]))
    Svec.append(Similarity(f,sel_basis[5]))
    #print(f)
    f.clear()
    SFec.append(Svec)

In [17]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[27., 28., 28.,  ...,  6.,  6.,  8.],
        [26., 33., 31.,  ...,  3.,  4.,  5.],
        [24., 24., 24.,  ...,  5., 10., 10.],
        ...,
        [35., 37., 38.,  ...,  6.,  3.,  4.],
        [33., 36., 34.,  ...,  2.,  3.,  1.],
        [22., 23., 22.,  ...,  2.,  3.,  6.]])


'torch.FloatTensor'

# W-GCN

In [18]:
data.x=CC_domain
print(data)

Data(x=[3327, 12], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [21]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.1037, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 02, Loss: 1.9891, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 03, Loss: 1.9702, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 04, Loss: 1.9092, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 05, Loss: 1.7767, Train: 16.67%, Valid: 17.20% Test: 18.10%
Run: 01, Epoch: 06, Loss: 1.7544, Train: 17.50%, Valid: 17.60% Test: 18.40%
Run: 01, Epoch: 07, Loss: 1.7320, Train: 20.00%, Valid: 18.00% Test: 18.80%
Run: 01, Epoch: 08, Loss: 1.6912, Train: 23.33%, Valid: 18.60% Test: 21.00%
Run: 01, Epoch: 09, Loss: 1.6717, Train: 26.67%, Valid: 18.80% Test: 21.10%
Run: 01, Epoch: 10, Loss: 1.6498, Train: 24.17%, Valid: 19.40% Test: 21.60%
Run: 01, Epoch: 11, Loss: 1.5722, Train: 25.00%, Valid: 20.00% Test: 21.70%
Run: 01, Epoch: 12, Loss: 1.5393, Train: 21.67%, Valid: 19.80% Test: 21.00%
Run: 01, Epoch: 13, Loss: 1.4960, Train: 20.00%, Valid: 19.40% Test: 20.70%
Run: 01, Epo

Run: 01, Epoch: 150, Loss: 0.5126, Train: 72.50%, Valid: 67.80% Test: 68.30%
Run: 01, Epoch: 151, Loss: 0.4237, Train: 68.33%, Valid: 63.60% Test: 63.40%
Run: 01, Epoch: 152, Loss: 0.3927, Train: 64.17%, Valid: 61.60% Test: 63.20%
Run: 01, Epoch: 153, Loss: 0.4358, Train: 70.00%, Valid: 64.00% Test: 65.30%
Run: 01, Epoch: 154, Loss: 0.4207, Train: 78.33%, Valid: 70.60% Test: 69.70%
Run: 01, Epoch: 155, Loss: 0.4024, Train: 79.17%, Valid: 70.60% Test: 72.80%
Run: 01, Epoch: 156, Loss: 0.4739, Train: 81.67%, Valid: 73.00% Test: 74.90%
Run: 01, Epoch: 157, Loss: 0.4814, Train: 77.50%, Valid: 74.00% Test: 74.20%
Run: 01, Epoch: 158, Loss: 0.3947, Train: 71.67%, Valid: 69.60% Test: 69.00%
Run: 01, Epoch: 159, Loss: 0.4992, Train: 67.50%, Valid: 67.20% Test: 66.00%
Run: 01, Epoch: 160, Loss: 0.4027, Train: 73.33%, Valid: 67.20% Test: 66.20%
Run: 01, Epoch: 161, Loss: 0.4652, Train: 77.50%, Valid: 66.60% Test: 67.90%
Run: 01, Epoch: 162, Loss: 0.3942, Train: 76.67%, Valid: 67.80% Test: 70.10%

Run: 02, Epoch: 123, Loss: 0.4419, Train: 77.50%, Valid: 75.80% Test: 75.90%
Run: 02, Epoch: 124, Loss: 0.4167, Train: 83.33%, Valid: 76.60% Test: 76.00%
Run: 02, Epoch: 125, Loss: 0.4887, Train: 85.83%, Valid: 75.00% Test: 75.80%
Run: 02, Epoch: 126, Loss: 0.4773, Train: 83.33%, Valid: 74.60% Test: 75.60%
Run: 02, Epoch: 127, Loss: 0.4500, Train: 83.33%, Valid: 74.20% Test: 74.80%
Run: 02, Epoch: 128, Loss: 0.5259, Train: 80.83%, Valid: 72.60% Test: 75.00%
Run: 02, Epoch: 129, Loss: 0.4569, Train: 77.50%, Valid: 70.20% Test: 73.10%
Run: 02, Epoch: 130, Loss: 0.5398, Train: 81.67%, Valid: 71.40% Test: 73.40%
Run: 02, Epoch: 131, Loss: 0.4620, Train: 84.17%, Valid: 71.00% Test: 71.90%
Run: 02, Epoch: 132, Loss: 0.4073, Train: 83.33%, Valid: 71.00% Test: 71.20%
Run: 02, Epoch: 133, Loss: 0.4688, Train: 80.83%, Valid: 70.00% Test: 70.60%
Run: 02, Epoch: 134, Loss: 0.3970, Train: 80.83%, Valid: 71.00% Test: 70.70%
Run: 02, Epoch: 135, Loss: 0.4834, Train: 85.00%, Valid: 74.20% Test: 74.70%

Run: 03, Epoch: 82, Loss: 0.6525, Train: 74.17%, Valid: 67.60% Test: 69.20%
Run: 03, Epoch: 83, Loss: 0.5805, Train: 77.50%, Valid: 71.40% Test: 70.70%
Run: 03, Epoch: 84, Loss: 0.5642, Train: 78.33%, Valid: 72.60% Test: 71.00%
Run: 03, Epoch: 85, Loss: 0.5752, Train: 70.00%, Valid: 69.40% Test: 68.70%
Run: 03, Epoch: 86, Loss: 0.6311, Train: 69.17%, Valid: 67.00% Test: 67.10%
Run: 03, Epoch: 87, Loss: 0.6121, Train: 73.33%, Valid: 71.00% Test: 70.30%
Run: 03, Epoch: 88, Loss: 0.5981, Train: 80.00%, Valid: 75.20% Test: 75.80%
Run: 03, Epoch: 89, Loss: 0.5948, Train: 80.00%, Valid: 70.80% Test: 73.20%
Run: 03, Epoch: 90, Loss: 0.4862, Train: 70.00%, Valid: 62.80% Test: 64.40%
Run: 03, Epoch: 91, Loss: 0.5836, Train: 70.00%, Valid: 62.40% Test: 63.80%
Run: 03, Epoch: 92, Loss: 0.6191, Train: 70.83%, Valid: 66.00% Test: 69.20%
Run: 03, Epoch: 93, Loss: 0.5558, Train: 85.00%, Valid: 74.40% Test: 75.00%
Run: 03, Epoch: 94, Loss: 0.5353, Train: 86.67%, Valid: 76.00% Test: 76.70%
Run: 03, Epo

Run: 04, Epoch: 42, Loss: 0.7970, Train: 71.67%, Valid: 70.40% Test: 72.30%
Run: 04, Epoch: 43, Loss: 0.7881, Train: 70.83%, Valid: 71.20% Test: 72.40%
Run: 04, Epoch: 44, Loss: 0.7242, Train: 72.50%, Valid: 71.80% Test: 73.90%
Run: 04, Epoch: 45, Loss: 0.7655, Train: 74.17%, Valid: 74.40% Test: 75.50%
Run: 04, Epoch: 46, Loss: 0.7946, Train: 77.50%, Valid: 73.40% Test: 76.40%
Run: 04, Epoch: 47, Loss: 0.7796, Train: 77.50%, Valid: 71.20% Test: 74.50%
Run: 04, Epoch: 48, Loss: 0.6504, Train: 72.50%, Valid: 68.00% Test: 69.70%
Run: 04, Epoch: 49, Loss: 0.7133, Train: 69.17%, Valid: 62.40% Test: 65.50%
Run: 04, Epoch: 50, Loss: 0.7635, Train: 70.00%, Valid: 61.60% Test: 64.90%
Run: 04, Epoch: 51, Loss: 0.7309, Train: 74.17%, Valid: 64.60% Test: 65.70%
Run: 04, Epoch: 52, Loss: 0.6903, Train: 72.50%, Valid: 66.00% Test: 68.30%
Run: 04, Epoch: 53, Loss: 0.6964, Train: 73.33%, Valid: 69.40% Test: 70.10%
Run: 04, Epoch: 54, Loss: 0.7477, Train: 70.00%, Valid: 71.60% Test: 71.30%
Run: 04, Epo

...

Run: 07, Epoch: 101, Loss: 0.6249, Train: 79.17%, Valid: 71.00% Test: 72.80%
Run: 07, Epoch: 102, Loss: 0.4715, Train: 80.00%, Valid: 73.20% Test: 73.50%
Run: 07, Epoch: 103, Loss: 0.5863, Train: 79.17%, Valid: 75.60% Test: 75.50%
Run: 07, Epoch: 104, Loss: 0.4926, Train: 81.67%, Valid: 75.80% Test: 76.80%
Run: 07, Epoch: 105, Loss: 0.5069, Train: 80.00%, Valid: 75.40% Test: 77.40%
Run: 07, Epoch: 106, Loss: 0.4850, Train: 85.83%, Valid: 75.40% Test: 77.20%
Run: 07, Epoch: 107, Loss: 0.4933, Train: 86.67%, Valid: 75.80% Test: 78.20%
Run: 07, Epoch: 108, Loss: 0.4767, Train: 85.00%, Valid: 75.60% Test: 78.30%
Run: 07, Epoch: 109, Loss: 0.4751, Train: 86.67%, Valid: 75.60% Test: 78.50%
Run: 07, Epoch: 110, Loss: 0.5182, Train: 85.00%, Valid: 77.00% Test: 78.20%
Run: 07, Epoch: 111, Loss: 0.4797, Train: 84.17%, Valid: 76.00% Test: 77.60%
Run: 07, Epoch: 112, Loss: 0.5409, Train: 85.83%, Valid: 75.60% Test: 76.90%
Run: 07, Epoch: 113, Loss: 0.5253, Train: 82.50%, Valid: 73.80% Test: 75.20%

Run: 08, Epoch: 68, Loss: 0.6883, Train: 72.50%, Valid: 64.00% Test: 66.50%
Run: 08, Epoch: 69, Loss: 0.6272, Train: 70.83%, Valid: 60.40% Test: 61.50%
Run: 08, Epoch: 70, Loss: 0.6456, Train: 69.17%, Valid: 58.20% Test: 60.30%
Run: 08, Epoch: 71, Loss: 0.6772, Train: 77.50%, Valid: 64.00% Test: 65.40%
Run: 08, Epoch: 72, Loss: 0.6691, Train: 78.33%, Valid: 69.60% Test: 70.60%
Run: 08, Epoch: 73, Loss: 0.6414, Train: 81.67%, Valid: 71.80% Test: 74.40%
Run: 08, Epoch: 74, Loss: 0.6650, Train: 83.33%, Valid: 76.20% Test: 76.20%
Run: 08, Epoch: 75, Loss: 0.5766, Train: 81.67%, Valid: 76.60% Test: 78.20%
Run: 08, Epoch: 76, Loss: 0.6598, Train: 83.33%, Valid: 77.60% Test: 78.20%
Run: 08, Epoch: 77, Loss: 0.6397, Train: 84.17%, Valid: 76.60% Test: 76.70%
Run: 08, Epoch: 78, Loss: 0.5852, Train: 85.00%, Valid: 75.00% Test: 75.70%
Run: 08, Epoch: 79, Loss: 0.6391, Train: 83.33%, Valid: 72.60% Test: 73.40%
Run: 08, Epoch: 80, Loss: 0.5744, Train: 83.33%, Valid: 73.40% Test: 73.90%
Run: 08, Epo

Run: 09, Epoch: 29, Loss: 0.8628, Train: 59.17%, Valid: 65.80% Test: 68.90%
Run: 09, Epoch: 30, Loss: 0.9488, Train: 60.83%, Valid: 67.80% Test: 70.30%
Run: 09, Epoch: 31, Loss: 0.9149, Train: 65.00%, Valid: 69.40% Test: 71.40%
Run: 09, Epoch: 32, Loss: 0.8973, Train: 73.33%, Valid: 67.60% Test: 69.50%
Run: 09, Epoch: 33, Loss: 0.8892, Train: 78.33%, Valid: 67.60% Test: 68.80%
Run: 09, Epoch: 34, Loss: 0.8789, Train: 78.33%, Valid: 67.20% Test: 68.00%
Run: 09, Epoch: 35, Loss: 0.8664, Train: 78.33%, Valid: 68.60% Test: 68.60%
Run: 09, Epoch: 36, Loss: 0.8119, Train: 77.50%, Valid: 69.60% Test: 69.80%
Run: 09, Epoch: 37, Loss: 0.8168, Train: 74.17%, Valid: 70.40% Test: 70.30%
Run: 09, Epoch: 38, Loss: 0.8468, Train: 73.33%, Valid: 72.40% Test: 71.20%
Run: 09, Epoch: 39, Loss: 0.7988, Train: 73.33%, Valid: 74.00% Test: 74.80%
Run: 09, Epoch: 40, Loss: 0.6771, Train: 81.67%, Valid: 73.80% Test: 75.70%
Run: 09, Epoch: 41, Loss: 0.8282, Train: 81.67%, Valid: 73.00% Test: 74.20%
Run: 09, Epo

Run: 09, Epoch: 194, Loss: 0.4439, Train: 72.50%, Valid: 70.20% Test: 69.80%
Run: 09, Epoch: 195, Loss: 0.2937, Train: 77.50%, Valid: 73.00% Test: 73.50%
Run: 09, Epoch: 196, Loss: 0.3471, Train: 87.50%, Valid: 76.20% Test: 76.60%
Run: 09, Epoch: 197, Loss: 0.4269, Train: 90.83%, Valid: 76.20% Test: 75.50%
Run: 09, Epoch: 198, Loss: 0.3371, Train: 83.33%, Valid: 70.40% Test: 71.90%
Run: 09, Epoch: 199, Loss: 0.3366, Train: 80.83%, Valid: 65.60% Test: 68.20%
Run: 09, Epoch: 200, Loss: 0.3223, Train: 75.00%, Valid: 61.00% Test: 65.60%
Run 09:
Highest Train: 90.83
Highest Valid: 81.00
  Final Train: 83.33
   Final Test: 78.20
Run: 10, Epoch: 01, Loss: 2.1052, Train: 16.67%, Valid: 19.20% Test: 17.20%
Run: 10, Epoch: 02, Loss: 1.9823, Train: 16.67%, Valid: 19.00% Test: 17.10%
Run: 10, Epoch: 03, Loss: 1.9536, Train: 16.67%, Valid: 19.00% Test: 17.10%
Run: 10, Epoch: 04, Loss: 1.8475, Train: 16.67%, Valid: 19.20% Test: 17.10%
Run: 10, Epoch: 05, Loss: 1.8569, Train: 16.67%, Valid: 19.40% Te

Run: 10, Epoch: 160, Loss: 0.4198, Train: 80.00%, Valid: 70.20% Test: 69.30%
Run: 10, Epoch: 161, Loss: 0.5423, Train: 80.83%, Valid: 70.00% Test: 70.40%
Run: 10, Epoch: 162, Loss: 0.4787, Train: 82.50%, Valid: 69.60% Test: 71.40%
Run: 10, Epoch: 163, Loss: 0.4898, Train: 86.67%, Valid: 72.80% Test: 73.50%
Run: 10, Epoch: 164, Loss: 0.4384, Train: 91.67%, Valid: 73.60% Test: 75.20%
Run: 10, Epoch: 165, Loss: 0.5028, Train: 90.83%, Valid: 74.00% Test: 75.10%
Run: 10, Epoch: 166, Loss: 0.5078, Train: 88.33%, Valid: 74.40% Test: 75.20%
Run: 10, Epoch: 167, Loss: 0.4867, Train: 87.50%, Valid: 74.60% Test: 75.80%
Run: 10, Epoch: 168, Loss: 0.4366, Train: 85.83%, Valid: 73.80% Test: 76.10%
Run: 10, Epoch: 169, Loss: 0.4620, Train: 86.67%, Valid: 73.20% Test: 75.70%
Run: 10, Epoch: 170, Loss: 0.5293, Train: 87.50%, Valid: 73.60% Test: 75.90%
Run: 10, Epoch: 171, Loss: 0.4374, Train: 88.33%, Valid: 73.80% Test: 76.90%
Run: 10, Epoch: 172, Loss: 0.4668, Train: 86.67%, Valid: 73.00% Test: 76.20%

# Topological Encodding 

In [4]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')
data = dataset[0]
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [5]:
print(data.edge_index.numpy())

[[ 628  158  486 ... 2820 1643   33]
 [   0    1    1 ... 3324 3325 3326]]


In [7]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [8]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [21]:
print(G.number_of_edges())

9104


In [9]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [10]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [24]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

0  |  48 

2  |  1331 

4  |  795 

6  |  438 

8  |  237 

10  |  139 

12  |  110 

14  |  61 

16  |  33 

18  |  24 

20  |  30 

22  |  19 

24  |  17 

26  |  4 

28  |  3 

30  |  9 

32  |  7 

34  |  4 

36  |  2 

38  |  2 

40  |  1 

42  |  2 

44  |  1 

46  |  2 

54  |  1 

56  |  1 

58  |  1 

60  |  1 

68  |  1 

70  |  1 

102  |  1 

198  |  1 



In [22]:
import pyflagser
Node_fil=[0,2,4,6,8,10,12,14,16,18,20,22,24,30,34,100]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 3326 (100%)

In [33]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [24]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [34]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[3327, 12], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104], topo=[3327, 32])


# TOPO-W-GCN

In [36]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        #return x.log_softmax(dim=-1)
        return x

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [40]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.3,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                 #     f'Valid: {100 * valid_acc:.2f}% '
                 #     f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run 01:
Highest Train: 95.83
Highest Valid: 78.20
  Final Train: 79.17
   Final Test: 78.00
Run 02:
Highest Train: 91.67
Highest Valid: 78.80
  Final Train: 78.33
   Final Test: 79.40
Run 03:
Highest Train: 90.83
Highest Valid: 79.00
  Final Train: 75.83
   Final Test: 79.10
Run 04:
Highest Train: 92.50
Highest Valid: 79.80
  Final Train: 76.67
   Final Test: 79.50
Run 05:
Highest Train: 90.83
Highest Valid: 78.80
  Final Train: 74.17
   Final Test: 78.20
Run 06:
Highest Train: 91.67
Highest Valid: 77.60
  Final Train: 86.67
   Final Test: 75.30
Run 07:
Highest Train: 92.50
Highest Valid: 78.00
  Final Train: 82.50
   Final Test: 78.80
Run 08:
Highest Train: 91.67
Highest Valid: 79.40
  Final Train: 69.17
   Final Test: 78.50
Run 09:
Highest Train: 92.50
Highest Valid: 77.80
  Final Train: 84.17
   Final Test: 78.20
Run 10:
Highest Train: 93.33
Highest Valid: 78.40
  Final Train: 74.17
   Final Test: 76.50
All runs:
Highest Train: 92.33 ± 1.46
Highest Valid: 78.58 ± 0.71
  Final Train:

# TOPO-GCN

In [ ]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

In [ ]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.3,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                 #     f'Valid: {100 * valid_acc:.2f}% '
                 #     f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()